# ApacheSpark - ML - Supervisado

<p><strong>Objetivo: </strong> El objetivo de este cuaderno es aplicar un Clasificador de Arbol de decisión sobre un conjunto de datos ejemplo:</p>

Los árboles de decisión son populares para las tareas de clasificación y regresión de aprendizaje automático. Los árboles de decisión se utilizan ampliamente ya que son fáciles de interpretar, manejan características categóricas, se extienden a la configuración de clasificación multiclase, no requieren normalización de características y pueden capturar no linealidades e interacciones de características.

## Importando librerias

In [0]:
# Se importan todas las librerías necesarias para el ejemplo
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

El paquete <b>libsvm</b> implementa la API de fuente de datos Spark SQL para cargar datos LIBSVM como DataFrame. El DataFrame cargado tiene dos columnas: <b>label</b> que contiene etiquetas almacenadas como doubles y <b>features</b> que contienen vectores de características almacenados como Vectors.

Para utilizar la fuente de datos LIBSVM, debe establecer "libsvm" como formato en DataFrameReader y, opcionalmente, especificar opciones, por ejemplo:

In [0]:
%fs ls /databricks-datasets/definitive-guide/data/

path,name,size
dbfs:/databricks-datasets/definitive-guide/data/activity-data/,activity-data/,0
dbfs:/databricks-datasets/definitive-guide/data/bike-data/,bike-data/,0
dbfs:/databricks-datasets/definitive-guide/data/binary-classification/,binary-classification/,0
dbfs:/databricks-datasets/definitive-guide/data/clustering/,clustering/,0
dbfs:/databricks-datasets/definitive-guide/data/flight-data/,flight-data/,0
dbfs:/databricks-datasets/definitive-guide/data/flight-data-hive/,flight-data-hive/,0
dbfs:/databricks-datasets/definitive-guide/data/multiclass-classification/,multiclass-classification/,0
dbfs:/databricks-datasets/definitive-guide/data/regression/,regression/,0
dbfs:/databricks-datasets/definitive-guide/data/retail-data/,retail-data/,0
dbfs:/databricks-datasets/definitive-guide/data/sample_libsvm_data.txt,sample_libsvm_data.txt,104736


## Crear dataframe con los datos

In [0]:
data = spark.read.format("libsvm").load("/databricks-datasets/definitive-guide/data/sample_libsvm_data.txt")

Se visualizan los datos, existen dos columnas, una con la categoría y otra con un vector con diferentes características. Se puede observar que hay dos categorías:

In [0]:
display(data)

label,features
0.0,"Map(vectorType -> sparse, length -> 692, indices -> List(127, 128, 129, 130, 131, 154, 155, 156, 157, 158, 159, 181, 182, 183, 184, 185, 186, 187, 188, 189, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 289, 290, 291, 292, 293, 294, 295, 296, 297, 300, 301, 302, 316, 317, 318, 319, 320, 321, 328, 329, 330, 343, 344, 345, 346, 347, 348, 349, 356, 357, 358, 371, 372, 373, 374, 384, 385, 386, 399, 400, 401, 412, 413, 414, 426, 427, 428, 429, 440, 441, 442, 454, 455, 456, 457, 466, 467, 468, 469, 470, 482, 483, 484, 493, 494, 495, 496, 497, 510, 511, 512, 520, 521, 522, 523, 538, 539, 540, 547, 548, 549, 550, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577, 578, 594, 595, 596, 597, 598, 599, 600, 601, 602, 603, 604, 622, 623, 624, 625, 626, 627, 628, 629, 630, 651, 652, 653, 654, 655, 656, 657), values -> List(51.0, 159.0, 253.0, 159.0, 50.0, 48.0, 238.0, 252.0, 252.0, 252.0, 237.0, 54.0, 227.0, 253.0, 252.0, 239.0, 233.0, 252.0, 57.0, 6.0, 10.0, 60.0, 224.0, 252.0, 253.0, 252.0, 202.0, 84.0, 252.0, 253.0, 122.0, 163.0, 252.0, 252.0, 252.0, 253.0, 252.0, 252.0, 96.0, 189.0, 253.0, 167.0, 51.0, 238.0, 253.0, 253.0, 190.0, 114.0, 253.0, 228.0, 47.0, 79.0, 255.0, 168.0, 48.0, 238.0, 252.0, 252.0, 179.0, 12.0, 75.0, 121.0, 21.0, 253.0, 243.0, 50.0, 38.0, 165.0, 253.0, 233.0, 208.0, 84.0, 253.0, 252.0, 165.0, 7.0, 178.0, 252.0, 240.0, 71.0, 19.0, 28.0, 253.0, 252.0, 195.0, 57.0, 252.0, 252.0, 63.0, 253.0, 252.0, 195.0, 198.0, 253.0, 190.0, 255.0, 253.0, 196.0, 76.0, 246.0, 252.0, 112.0, 253.0, 252.0, 148.0, 85.0, 252.0, 230.0, 25.0, 7.0, 135.0, 253.0, 186.0, 12.0, 85.0, 252.0, 223.0, 7.0, 131.0, 252.0, 225.0, 71.0, 85.0, 252.0, 145.0, 48.0, 165.0, 252.0, 173.0, 86.0, 253.0, 225.0, 114.0, 238.0, 253.0, 162.0, 85.0, 252.0, 249.0, 146.0, 48.0, 29.0, 85.0, 178.0, 225.0, 253.0, 223.0, 167.0, 56.0, 85.0, 252.0, 252.0, 252.0, 229.0, 215.0, 252.0, 252.0, 252.0, 196.0, 130.0, 28.0, 199.0, 252.0, 252.0, 253.0, 252.0, 252.0, 233.0, 145.0, 25.0, 128.0, 252.0, 253.0, 252.0, 141.0, 37.0))"
1.0,"Map(vectorType -> sparse, length -> 692, indices -> List(158, 159, 160, 161, 185, 186, 187, 188, 189, 213, 214, 215, 216, 217, 240, 241, 242, 243, 244, 245, 267, 268, 269, 270, 271, 295, 296, 297, 298, 322, 323, 324, 325, 326, 349, 350, 351, 352, 353, 377, 378, 379, 380, 381, 404, 405, 406, 407, 408, 431, 432, 433, 434, 435, 459, 460, 461, 462, 463, 486, 487, 488, 489, 490, 514, 515, 516, 517, 518, 542, 543, 544, 545, 569, 570, 571, 572, 573, 596, 597, 598, 599, 600, 601, 624, 625, 626, 627, 652, 653, 654, 655, 680, 681, 682, 683), values -> List(124.0, 253.0, 255.0, 63.0, 96.0, 244.0, 251.0, 253.0, 62.0, 127.0, 251.0, 251.0, 253.0, 62.0, 68.0, 236.0, 251.0, 211.0, 31.0, 8.0, 60.0, 228.0, 251.0, 251.0, 94.0, 155.0, 253.0, 253.0, 189.0, 20.0, 253.0, 251.0, 235.0, 66.0, 32.0, 205.0, 253.0, 251.0, 126.0, 104.0, 251.0, 253.0, 184.0, 15.0, 80.0, 240.0, 251.0, 193.0, 23.0, 32.0, 253.0, 253.0, 253.0, 159.0, 151.0, 251.0, 251.0, 251.0, 39.0, 48.0, 221.0, 251.0, 251.0, 172.0, 234.0, 251.0, 251.0, 196.0, 12.0, 253.0, 251.0, 251.0, 89.0, 159.0, 255.0, 253.0, 253.0, 31.0, 48.0, 228.0, 253.0, 247.0, 140.0, 8.0, 64.0, 251.0, 253.0, 220.0, 64.0, 251.0, 253.0, 220.0, 24.0, 193.0, 253.0, 220.0))"
1.0,"Map(vectorType -> sparse, length -> 692, indices -> List(124, 125, 126, 127, 151, 152, 153, 154, 155, 179, 180, 181, 182, 183, 208, 209, 210, 211, 235, 236, 237, 238, 239, 263, 264, 265, 266, 267, 268, 292, 293, 294, 295, 296, 321, 322, 323, 324, 349, 350, 351, 352, 377, 378, 379, 380, 405, 406, 407, 408, 433, 434, 435, 436, 461, 462, 463, 464, 489, 490, 491, 492, 493, 517, 518, 519, 520, 521, 545, 546, 547, 548, 549, 574, 575, 576, 577, 578, 602, 603, 604, 605, 606, 630, 631, 632, 633, 634, 658, 659, 660, 661, 662), values -> List(145.0, 255.0, 211.0, 31.0, 32.0, 237.0, 253.0, 252.0, 71.0, 11.0, 175.0, 253.0, 252.0, 71.0, 144

## Utilizar transformadores

Se utilizan dos transformadores de características para preparar los datos, <b>StringIndexer</b> y <b>VectorIndexer</b>.

Esto ayuda a indexar categorías para la etiqueta(label) y las características categóricas(features), agregando metadatos al DataFrame que el algoritmo del árbol de decisiones puede reconocer.

Los métodos basados en árboles pueden manejar variables categóricas de forma natural. En spark.ml, solo necesita pasar las columnas categóricas al StringIndexer, y el árbol de decisiones puede encargarse del resto.

In [0]:
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(data)
# We specify maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)

## Dividir conjunto de datos en entrenamiento y prueba

Se divide el conjunto de datos en dos, un 70% para entrenar el algoritmo y un 30% para probar el modelo obtenido:

In [0]:
(trainingData, testData) = data.randomSplit([0.7, 0.3])

## Configurar el modelo

En este paso se configura el modelo de Arbol de Decision, se le especifican la variable objetivo <b>labelCol="indexedLabel"</b> que debe ser un double siempre, y las variables para entrenar el modelo estarán en <b>featuresCol="indexedFeatures"</b> que debe ser siempre de tipo Vector.

In [0]:
dt = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")

Se puede utilizar la función <b>explainParams</b> para revisar todos los parámetros que se le pueden configurar al clasificador:

In [0]:
print(dt.explainParams())

cacheNodeIds: If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval. (default: False)
checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext. (default: 10)
featuresCol: features column name. (default: features, current: indexedFeatures)
impurity: Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gini (default: gini)
labelCol: label column name. (default: label, current: indexedLabel)
leafCol: Leaf indices column name. Predicted leaf index of each instance in each tree by preorder. (default: )
maxBins: Max number of bin

## Ejecutar Pipeline

Aquí se utiliza un Pipeline para enlazar las transformadores y el modelo:

In [0]:
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, dt])

En este paso se ejecuta el Pipeline, se ejecutan los transformadores, y luego se entrena el modelo con los datos que se pasan por parámetro. Se pasa por parámetro el conjunto de entrenamiento:

In [0]:
model = pipeline.fit(trainingData)

## Realizar predicciones

En este paso se realizan predicciones utilizando el conjunto de prueba

In [0]:
# realizar predicciones
predictions = model.transform(testData)

Se muestran las columnas para comparar predicción con realidad:

In [0]:
# Seleccionar algunos ejemplos para mostrar
predictions.select("prediction", "indexedLabel", "features").show(5)

+----------+------------+--------------------+
|prediction|indexedLabel|            features|
+----------+------------+--------------------+
|       1.0|         1.0|(692,[95,96,97,12...|
|       1.0|         1.0|(692,[98,99,100,1...|
|       1.0|         1.0|(692,[100,101,102...|
|       1.0|         1.0|(692,[123,124,125...|
|       1.0|         1.0|(692,[123,124,125...|
+----------+------------+--------------------+
only showing top 5 rows



## Evaluar el modelo

Para evaluar el modelo se utiliza <b>MulticlassClassificationEvaluator</b> que recibe por parámetro, la variable objetivo (labelCol), la variable predictora que se calculó previamente (predictionCol) y la métrica que se desea calcular, en este caso Accuracy.

Tambien permite calcular todas estas: f1|accuracy|weightedPrecision|weightedRecall|weightedTruePositiveRate| weightedFalsePositiveRate|weightedFMeasure|truePositiveRateByLabel| falsePositiveRateByLabel|precisionByLabel|recallByLabel|fMeasureByLabel| logLoss|hammingLoss

In [0]:
evaluator = MulticlassClassificationEvaluator(labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g " % (accuracy))

Test Error = 1 


In [0]:
evaluator = MulticlassClassificationEvaluator(labelCol="indexedLabel", predictionCol="prediction", metricName="precisionByLabel")
precision = evaluator.evaluate(predictions)
print("Test Error = %g " % (precision))

Test Error = 1 


In [0]:
evaluator = MulticlassClassificationEvaluator(labelCol="indexedLabel", predictionCol="prediction", metricName="recallByLabel")
recall = evaluator.evaluate(predictions)
print("Test Error = %g " % (recall))

Test Error = 1 
